In [1]:
import pandas as pd

# Load the CSV file
csv_file = '2nd24hrs_mpma.csv'
df = pd.read_csv(csv_file)

# Drop all columns except the specified ones
columns_to_keep = ['Week Number', 'Seconds of Week', 'PRN', 'Amplitude']
df = df[columns_to_keep]

# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude
0          260     287020.28125   11   0.022537
1          260     287020.06250    6   0.032610
2          260     287020.31250   13   0.049030
3          260     287020.09375   14   0.052059
4          260     287020.31250   17   0.082222

Tail of the file:
        Week Number  Seconds of Week  PRN  Amplitude
899321          260     373424.31250    6   0.095259
899322          260     373424.34375   12   0.077140
899323          260     373424.34375   13   0.126194
899324          260     373424.31250   22   0.047479
899325          260     373424.34375   17   0.070234


In [2]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df['UTC Time'] = df.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df['24-hour Time'] = df['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df.head())


Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN  Amplitude                   UTC Time  \
0          260     287020.28125   11   0.022537 1985-01-02 07:43:40.281250   
1          260     287020.06250    6   0.032610 1985-01-02 07:43:40.062500   
2          260     287020.31250   13   0.049030 1985-01-02 07:43:40.312500   
3          260     287020.09375   14   0.052059 1985-01-02 07:43:40.093750   
4          260     287020.31250   17   0.082222 1985-01-02 07:43:40.312500   

  24-hour Time  
0     07:43:40  
1     07:43:40  
2     07:43:40  
3     07:43:40  
4     07:43:40  


In [3]:
# Round the "Seconds of Week" column to the nearest integer
#df['Seconds of Week'] = df['Seconds of Week'].round().astype(int)

# Normalize the "Seconds of Week" column by subtracting the first value
#first_seconds = df['Seconds of Week'].iloc[0]
#df['Seconds of Week'] = df['Seconds of Week'] - first_seconds

# Print the first few rows
#print("Head of the file:")
#print(df.head())

# Print the last few rows
#print("\nTail of the file:")
#print(df.tail())

In [4]:
import numpy as np
# Replace zero or very small values of "Amplitude" to avoid log10 issues
df['Amplitude'] = df['Amplitude'].replace(0, np.nan)
df['Amplitude'] = df['Amplitude'].fillna(df['Amplitude'].min() / 2)

# Add a new column 'd/u' which is 20log10(Amplitude)
df['d/u'] = -1*20 * np.log10(df['Amplitude'])

# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude                   UTC Time  \
0          260     287020.28125   11   0.022537 1985-01-02 07:43:40.281250   
1          260     287020.06250    6   0.032610 1985-01-02 07:43:40.062500   
2          260     287020.31250   13   0.049030 1985-01-02 07:43:40.312500   
3          260     287020.09375   14   0.052059 1985-01-02 07:43:40.093750   
4          260     287020.31250   17   0.082222 1985-01-02 07:43:40.312500   

  24-hour Time        d/u  
0     07:43:40  32.942083  
1     07:43:40  29.733095  
2     07:43:40  26.190771  
3     07:43:40  25.670134  
4     07:43:40  21.700229  

Tail of the file:
        Week Number  Seconds of Week  PRN  Amplitude  \
899321          260     373424.31250    6   0.095259   
899322          260     373424.34375   12   0.077140   
899323          260     373424.34375   13   0.126194   
899324          260     373424.31250   22   0.047479   
899325          260     373424.34375   17   0.

In [5]:
import plotly.io as pio
pio.renderers.default = "iframe"
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN
for prn in df['PRN'].unique():
    group = df[df['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['d/u'], mode='lines', name=f'PRN {prn}'))

# Update layout
fig.update_layout(
    title='d/u values for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='d/u (dB)',
    xaxis_tickformat='%H:%M:%S'
)

# Show the figure
fig.show()


In [6]:
grouped = df.groupby('PRN')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())


PRN: 2
Head of the group:
        Week Number  Seconds of Week  PRN  Amplitude  \
370464          260     323752.84375    2   0.099433   
370475          260     323753.84375    2   0.085743   
370486          260     323754.84375    2   0.141377   
370497          260     323755.84375    2   0.107538   
370508          260     323756.84375    2   0.111225   

                         UTC Time 24-hour Time        d/u  
370464 1985-01-02 17:55:52.843750     17:55:52  20.049405  
370475 1985-01-02 17:55:53.843750     17:55:53  21.336036  
370486 1985-01-02 17:55:54.843750     17:55:54  16.992421  
370497 1985-01-02 17:55:55.843750     17:55:55  19.368772  
370508 1985-01-02 17:55:56.843750     17:55:56  19.075913  

Tail of the group:
        Week Number  Seconds of Week  PRN  Amplitude  \
814200          260     365602.34375    2   0.092985   
814212          260     365603.34375    2   0.175911   
814223          260     365604.34375    2   0.115762   
814234          260     365605.3

In [7]:
%matplotlib widget
import matplotlib.pyplot as plt

# Ask user to input a specific PRN
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    plt.figure(figsize=(12, 6))
    plt.plot(group['24-hour Time'], group['d/u'], marker='o', linestyle='-', color='b')
    plt.title(f'd/u values for PRN {specific_prn}')
    plt.xlabel('Time (24-hour format)')
    plt.ylabel('d/u (dB)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")

Enter a PRN to plot (e.g., 32):  99


PRN 99 not found in the dataset.


In [8]:
import plotly.io as pio
pio.renderers.default = "iframe"


In [9]:
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    fig = px.line(group, x='24-hour Time', y='d/u', title=f'd/u values for PRN {specific_prn}')
    fig.update_layout(
        xaxis_title='Time (24-hour format)',
        yaxis_title='d/u (dB)',
        xaxis_tickformat='%H:%M:%S'
    )
    fig.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")


Enter a PRN to plot (e.g., 32):  99


PRN 99 not found in the dataset.


### 

In [10]:
# Count the number of values below 16
count_below_16 = df["d/u"][df["d/u"] < 16].count()

# Calculate the total number of values in the column
total_values = df["d/u"].count()

# Calculate the percentage of values below 16
percentage_below_16 = (count_below_16 / total_values) * 100

# Count the number of values below 20
count_below_20= df["d/u"][df["d/u"] < 20].count()

# Calculate the total number of values in the column
total_values = df["d/u"].count()

# Calculate the percentage of values below 16
percentage_below_20 = (count_below_20 / total_values) * 100

# Count the number of values >= 20
count_eqgt_20= df["d/u"][df["d/u"] >= 20].count()

# Calculate the total number of values in the column
total_values = df["d/u"].count()

# Calculate the percentage of values below 16
percentage_eqgt_20 = (count_eqgt_20 / total_values) * 100
print(f"Percentage of d/u values below 16: {percentage_below_16:.2f}%")
print(f"Percentage of d/u values below 20: {percentage_below_20:.2f}%")
print(f"Percentage of d/u values >= 20: {percentage_eqgt_20:.2f}%")




Percentage of d/u values below 16: 2.19%
Percentage of d/u values below 20: 11.11%
Percentage of d/u values >= 20: 88.89%
